In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import ta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from binance.client import Client

# Autenticación en la API de Binance
api_key = 'GFFbe5wk9HnG4eqS1Wlw54WkHrEExQ11yslR3qx6fFTapoXfvHSgcixcRLrB7Cgy'
api_secret = 'lUJlWrU7EhHR3YdcbTFqtdCwYc8qIw5NRUhqYATQ7DtjujYSO9nLZrPXINsvMdLc'
client = Client(api_key, api_secret)

# Obtención de los datos históricos de precios de Bitcoin
klines = client.futures_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1HOUR, "1 year ago UTC")

# Creación de un dataframe con los datos
data = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])

# Conversión de los precios y volumen a float
data['open'] = data['open'].astype(float)
data['high'] = data['high'].astype(float)
data['low'] = data['low'].astype(float)
data['close'] = data['close'].astype(float)
data['volume'] = data['volume'].astype(float)

# Normalizar los datos
scaler = MinMaxScaler()
data_scaled = pd.DataFrame(scaler.fit_transform(data[['open', 'high', 'low', 'close', 'volume']]), columns=['open', 'high', 'low', 'close', 'volume'])

# Procesar los datos para la entrada del modelo
data_scaled['t-24'] = data_scaled['close'].shift(24)

# Calcular indicadores técnicos
data_scaled['RSI'] = ta.momentum.RSIIndicator(data_scaled['close']).rsi()
data_scaled['MACD'] = ta.trend.MACD(data_scaled['close']).macd_diff()
data_scaled['EMA_100'] = ta.trend.EMAIndicator(data_scaled['close'], 100).ema_indicator()
data_scaled['EMA_200'] = ta.trend.EMAIndicator(data_scaled['close'], 200).ema_indicator()
data_scaled['ADX'] = ta.trend.ADXIndicator(data_scaled['high'], data_scaled['low'], data_scaled['close']).adx()
data_scaled['ATR'] = ta.volatility.AverageTrueRange(data_scaled['high'], data_scaled['low'], data_scaled['close']).average_true_range()
data_scaled['Momentum'] = ta.momentum.AwesomeOscillatorIndicator(data_scaled['high'], data_scaled['low']).awesome_oscillator()
data_scaled['Volume'] = data_scaled['volume']

# Eliminar las filas con valores NaN
data_scaled.dropna(inplace=True)

X = np.array(data_scaled[['close', 't-24', 'RSI', 'MACD', 'EMA_100', 'EMA_200', 'ADX', 'ATR', 'Momentum', 'Volume']])
X = np.reshape(X, (X.shape[0], X.shape[1], 1))
y = np.array(data_scaled['close'])

# Dividir los datos en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el modelo de red neuronal
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(100, return_sequences=True, input_shape=(X.shape[1], 1)),  # Aumenté las unidades en la LSTM
    tf.keras.layers.LSTM(50, return_sequences=False),  # Añadí una segunda capa LSTM
    tf.keras.layers.Dense(50, activation='relu'),  # Añadí una capa densa intermedia
    tf.keras.layers.Dense(1)
])

# Compilar el modelo
model.compile(loss='mse', optimizer='adam')

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_val, y_val))  # Aumenté las epochs y ajusté el tamaño del batch

# Guardar el modelo entrenado
model.save('modelos de entrenamiento/btc_price_prediction_model_indicators.h5')

/home/samueltg92/anaconda3/envs/algotrading/lib/python3.10/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/home/samueltg92/anaconda3/envs/algotrading/lib/python3.10/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
2023-05-13 10:12:16.115772: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-13 10:12:16.116078: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registe

Epoch 1/200


2023-05-13 10:12:16.352340: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-13 10:12:16.353485: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-13 10:12:16.354174: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

207/214 [============================>.] - ETA: 0s - loss: 0.0259

2023-05-13 10:12:19.268616: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-13 10:12:19.269742: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-13 10:12:19.270617: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

214/214 [==============================] - 3s 8ms/step - loss: 0.0251 - val_loss: 8.8601e-04
Epoch 2/200
214/214 [==============================] - 1s 6ms/step - loss: 8.5264e-04 - val_loss: 8.1909e-04
Epoch 3/200
214/214 [==============================] - 1s 6ms/step - loss: 7.4289e-04 - val_loss: 7.5732e-04
Epoch 4/200
214/214 [==============================] - 1s 6ms/step - loss: 6.7581e-04 - val_loss: 5.4210e-04
Epoch 5/200
214/214 [==============================] - 1s 6ms/step - loss: 5.2988e-04 - val_loss: 4.0301e-04
Epoch 6/200
214/214 [==============================] - 1s 6ms/step - loss: 4.9327e-04 - val_loss: 3.8047e-04
Epoch 7/200
214/214 [==============================] - 1s 6ms/step - loss: 4.3153e-04 - val_loss: 4.2722e-04
Epoch 8/200
214/214 [==============================] - 1s 6ms/step - loss: 4.3094e-04 - val_loss: 3.0102e-04
Epoch 9/200
214/214 [==============================] - 1s 6ms/step - loss: 3.9173e-04 - val_loss: 3.2216e-04
Epoch 10/200
214/214 [=============